# Tutorial SEGOMOE via web services

This notebook describes the usage of segomoe using web services exposed by WhatsOpt.

## Prerequisite

### wop (WhatsOpt command line interface)

You have to install the `wop` command:

```bash
pip install -U wop
```

You need to be logged in a WhatsOpt server either by using the <code>wop</code> command in a shell or the WhatsOpt API as follows:

In [1]:
from whatsopt.whatsopt_client import WhatsOpt
wop = WhatsOpt(url="https://ether.onera.fr/whatsopt")
ok = wop.login(echo=True)

Successfully logged in to remote WhatsOpt https://selene.onecert.fr/whatsopt



To test if you are connected the following command should succeed

In [2]:
wop.check_versions()

WhatsOpt 1.30.0 requires wop >= 1.20.0
You are using wop 2.5.5


### SMT (Surrogate Modeling Toolbox)

SMT is not required per se, but is used in this notebook to get optimized LHS sampling method.

```bash
pip install smt
```

## Optimization without constraint

### Objective function

First we define the objective function we want to minimize

In [3]:
import numpy as np

def f_obj(x):
    """
    Function Six-Hump Camel Back
    2 global optimum y_opt =-1.0316 located at x_opt = (0.089842, -0.712656) or (-0.089842, 0.712656)
    https://www.sfu.ca/~ssurjano/camel6.html
    """
    x_ = np.atleast_2d(x)
    x1 = np.array(x_)[:, 0]
    x2 = np.array(x_)[:, 1]
    val = 4*x1**2-2.1*x1**4+1./3.*x1**6+x1*x2-4*x2**2+4*x2**4
    return np.atleast_2d(val).T

### Optimization

First we create an optimization context to use the SEGOMOE optimizer with the design space <code>xlimits</code>.

In [4]:
from whatsopt.optimization import Optimization

xlimits = [[-3, 3], [-2, 2]]
optim = Optimization(xlimits, options={"mod_obj__regr": "constant", "optimizer": "cobyla"})

We need to have an initial DOE (n_samples, nx) and the corresponding outputs y (n_samples, 1).

In [5]:
import numpy as np
# from smt.sampling_methods import LHS
# lhs = LHS(xlimits=np.array(xlimits), criterion='ese')
# xdoe = lhs(5)

xdoe = np.array([[-0.91502224,  1.89017506],
 [ 2.57253436,  0.83786997],
 [-2.32304511, -1.12821447],
 [ 1.65152355, -1.63067708],
 [ 0.0057155,  -0.31036381]])
ydoe = f_obj(xdoe)
print("Initial DOE")
print("xdoe={}".format(xdoe))
print("ydoe={}".format(ydoe))

Initial DOE
xdoe=[[-0.91502224  1.89017506]
 [ 2.57253436  0.83786997]
 [-2.32304511 -1.12821447]
 [ 1.65152355 -1.63067708]
 [ 0.0057155  -0.31036381]]
ydoe=[[37.11048572]
 [32.43194792]
 [16.82595609]
 [17.00496851]
 [-0.34983144]]


We initialize the optimizer with the inital DOE.

In [6]:
optim.tell_doe(xdoe, ydoe)

We trigger the optimization using the "ask and tell" interface.

In [7]:
# We loop using the iteration budget
n_iter = 1
for i in range(n_iter):
    x_suggested, status, x_best, y_best = optim.ask()
    print("{} x suggested = {} with status: {}".format(i, x_suggested, Optimization.STATUSES[status]))

    # compute objective function at the suggested point
    new_y = f_obj(x_suggested)
    print("new y = {}".format(new_y))

    optim.tell(x_suggested, new_y)
    if optim.is_solution_reached():
        print("Solution is reached")
        break

x_opt, y_opt = optim.get_result()
print(f"Found minimum y_opt = {y_opt} at x_opt = {x_opt}")

0 x suggested = [0.6569666173623493, -0.3805074037288991] with status: pending
new y = [[0.6167541]]
Found minimum y_opt = [[-0.34983144]] at x_opt = [[ 0.0057155  -0.31036381]]


In [8]:
optim.get_result()

(array([[ 0.0057155 , -0.31036381]]), array([[-0.34983144]]))

In [9]:
optim.get_history()

(array([[-0.91502224,  1.89017506],
        [ 2.57253436,  0.83786997],
        [-2.32304511, -1.12821447],
        [ 1.65152355, -1.63067708],
        [ 0.0057155 , -0.31036381],
        [ 0.65696662, -0.3805074 ]]),
 array([[37.11048572],
        [32.43194792],
        [16.82595609],
        [17.00496851],
        [-0.34983144],
        [ 0.6167541 ]]))

For convenience, the previous optimization loop is available as the <code>run</code> method of the optimization object.

In [10]:
# to reset the initial DOE, otherwise optimization will go on from previous state
optim.tell_doe(xdoe, ydoe)

# run the optimization loop again
optim.run(f_obj, 15)

0 x suggested = [0.6569656661437774, -0.380507500727063] with status: pending
new y = [[0.61675122]]
1 x suggested = [-0.7155110447997332, -0.3242126449938182] with status: pending
new y = [[1.39786209]]
2 x suggested = [0.05876967977330048, -0.4721529010461281] with status: pending
new y = [[-0.70688304]]
3 x suggested = [0.07165374464285135, -0.4770547137390563] with status: pending
new y = [[-0.71685302]]
4 x suggested = [0.176210768034101, -0.5108365742185258] with status: pending
new y = [[-0.73925668]]
5 x suggested = [0.1374141522708519, -0.5192673495923205] with status: pending
new y = [[-0.78430507]]
6 x suggested = [0.0008375686109762055, -0.7890770033050413] with status: pending
new y = [[-0.94049335]]
7 x suggested = [0.02437176953276468, -0.6964257656715734] with status: pending
new y = [[-1.013699]]
8 x suggested = [0.04906193142278131, -0.699058831032127] with status: pending
new y = [[-1.02416877]]
9 x suggested = [0.08000685620908185, -0.7106182418464171] with status: 

(array([[ 0.08979346, -0.71263142]]), array([[-1.03162844]]))

## Optimization with constraints

### Objective and constraints functions

We define objective and constraints function and we build a grouped function which allows to evaluate all in one go.

In [11]:
import numpy as np

# Objective
def G24(point):
    """
    Function G24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def f_grouped(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T

In [12]:
f_grouped([1, 2])

array([[-3., -2.,  2.]])

### Optimization

First we create an optimization context to use the SEGOMOE optimizer with the design space <code>xlimits</code> and the constraints specifications <code>cstr_specs</code>.

In [13]:
from whatsopt.optimization import Optimization

xlimits = [[0, 3], [0, 4]]
cstr_specs = 2*[{"type": '<', "bound": 0.0}]
optim = Optimization(xlimits, cstr_specs)

Constraint type can be iether <code><</code>, <code>=</code> or <code>></code>. A tolerance may be specified using the <code>tol</code> key. Constraints defaults are :

In [14]:
print(Optimization.DEFAULT_CSTR)

{'type': '<', 'bound': 0.0, 'tol': 0.0001}


In [15]:
import numpy as np
from smt.sampling_methods import LHS
lhs = LHS(xlimits=np.array(xlimits), criterion='ese')

xdoe = lhs(5)
ydoe = f_grouped(xdoe)
print("Initial DOE")
print("xdoe={}".format(xdoe))
print("ydoe={}".format(ydoe))

Initial DOE
xdoe=[[1.2814057  0.37895828]
 [1.88375454 1.99871634]
 [0.27592919 1.58883291]
 [2.84710887 2.87094557]
 [0.670875   3.31379397]]
ydoe=[[ -1.66036398  -3.3168269   -0.55660358]
 [ -3.88247088  -0.09718631  -1.89391001]
 [ -1.8647621   -0.86378891 -13.97293212]
 [ -5.71805444 -10.76272085   2.55193171]
 [ -3.98466897  -0.27638083   0.96325569]]


In [16]:
optim.tell_doe(xdoe, ydoe)

In [17]:
optim.run(f_grouped, n_iter=20, with_best=True)

0 x suggested = [3.0, 4.0] with status: pending
x_best=[[1.8837545418465553, 1.9987163420721328]]
y_best=[[-3.882470883918688]]
new y = [[ -7. -16.   4.]]
1 x suggested = [2.6339276698424148, 4.0] with status: pending
x_best=[[1.8837545418465553, 1.9987163420721328]]
y_best=[[-3.882470883918688]]
new y = [[-6.63392767 -3.57592729  2.56893469]]
2 x suggested = [1.9619938323210693, 4.0] with status: pending
x_best=[[1.8837545418465553, 1.9987163420721328]]
y_best=[[-3.882470883918688]]
new y = [[-5.96199383  1.98887927  0.0115474 ]]
3 x suggested = [1.7541979688308198, 3.9999999999999996] with status: pending
x_best=[[1.8837545418465553, 1.9987163420721328]]
y_best=[[-3.882470883918688]]
new y = [[-5.75419797  1.62815826  0.46874746]]
4 x suggested = [1.6164881524304522, 3.129226885043262] with status: pending
x_best=[[1.8837545418465553, 1.9987163420721328]]
y_best=[[-3.882470883918688]]
new y = [[-4.74571504  0.36056983  0.2193459 ]]
5 x suggested = [2.4609749898836544, 2.3232655479226

(array([[2.32952022, 3.17849836]]), array([[-5.50801858]]))

Note: if the suggested x has already been told or very close a previous suggestion (see <code>numpy.allclose</code>), the solution is considered to be reached.

In [18]:
optim.get_status()

'pending'

In [19]:
optim.get_history()

(array([[1.2814057 , 0.37895828],
        [1.88375454, 1.99871634],
        [0.27592919, 1.58883291],
        [2.84710887, 2.87094557],
        [0.670875  , 3.31379397],
        [3.        , 4.        ],
        [2.63392767, 4.        ],
        [1.96199383, 4.        ],
        [1.75419797, 4.        ],
        [1.61648815, 3.12922689],
        [2.46097499, 2.32326555],
        [2.25743575, 2.91588889],
        [2.34704474, 3.03413423],
        [2.32991313, 3.15691111],
        [2.3295487 , 3.17821002],
        [2.32951939, 3.17849606],
        [2.32952017, 3.17849301],
        [2.32952022, 3.17849836],
        [1.5843741 , 2.8892666 ],
        [1.59610848, 2.79516066],
        [1.59988623, 2.81991254],
        [2.32952057, 3.17849565],
        [1.59962054, 2.82036433],
        [1.59962128, 2.82036352],
        [0.26155349, 4.        ]]),
 array([[-1.66036398e+00, -3.31682690e+00, -5.56603581e-01],
        [-3.88247088e+00, -9.71863079e-02, -1.89391001e+00],
        [-1.86476210e+00, 